# Dynamic Modification of a ProgressiVis Program

This notebook shows how a running ProgressiVis program can be modified.
A simple program is created that loads a CSV file, compute the minumum, and prints a dot for each chunk.
Then, the program is modified; a max module is added after the CSV module and a slash is printed at each chunk.
Finally, the initial min module is deleted, triggering the deletion of the first print a **collateral dammage**.

In [ ]:
from progressivis import (
    CSVLoader,
    Print,
    Min, Max,
)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
LARGE_TAXI_FILE = "https://www.aviz.fr/nyc-taxi/yellow_tripdata_2015-01.csv.bz2"

# prints a dot at each chunk (run_step)
def terse(x):
    print(".", end="", flush=True)

# prints a slash at each chunk
def terse2(x):
    print("/", end="", flush=True)

## Creates the initial program

The print module calls the function `terse` at each chunk, showing a dot when run.

In [ ]:
csv = CSVLoader(LARGE_TAXI_FILE, usecols=['pickup_longitude', 'pickup_latitude'])
m = Min(name="min")
prt = Print(proc=terse)
m.input.table = csv.output.result
prt.input.df = m.output.result

In [ ]:
csv.scheduler.task_start()

In [ ]:
csv.scheduler

# Adds a branch

Compute the max and prints each chunk with a slash.
The branch is validated at the end of the `with` construct and, if valid, is run. Otherwise, an exception is raised and the initial program is not modified.

In [ ]:
with csv.scheduler as dataflow:
    M = Max(name="max")
    prt2 = Print(proc=terse2)
    M.input.table = csv.output.result
    prt2.input.df = M.output.result

In [ ]:
csv.scheduler

# Removes the initial branch

The min module is deleted, which also deletes the first print module.

In [ ]:
with csv.scheduler as dataflow:
    deps = dataflow.collateral_damage("min")
    print("The collateral damage of deleting min is:", deps)
    dataflow.delete_modules(*deps)

In [ ]:
csv.scheduler

In [ ]:
# csv.scheduler.task_stop()